In [7]:
from library_dicom.dicom_processor.tools.flip import *
from library_dicom.dicom_processor.tools.create_mip import *

In [13]:

csv_flip_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/petIni.csv'

with open(csv_flip_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [14]:
csv_data[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 '1',
 'cr_014_jul 25_2014.csv',
 '1',
 '5',
 '11.13',
 'Bone Marrow',
 '3.4',
 'Abdominal',
 '2.19',
 '3.05',
 '4.08',
 '2.61',
 '3.35',
 '1',
 '6.7',
 '3.46',
 '4',
 '3.54',
 '2']

In [4]:
for serie in csv_data:
    print(csv_data.index(serie))
    roi, suv_max = get_suv_max_value(serie[6], serie[5])
    serie.append(roi)
    serie.append(suv_max)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [5]:
csv_data[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 'pet0',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_mask.nii',
 19,
 9.66]

In [15]:
csv_flip_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/FLIP_SUV_MAX_BIGGER_ROI.csv'

with open(csv_flip_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data_2 = []
    for row in reader :
        csv_data_2.append(row)
        
del csv_data_2[0] #enlever première ligne


In [16]:
csv_data_2[1]

['CR,017',
 '2017017',
 '1.2.840.113619.2.354.3.3523896338.523.1419230918.308',
 'pet0',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,017/pet0/1.2.840.113619.2.354.3.3523896338.523.1419230918.308_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,017/pet0/1.2.840.113619.2.354.3.3523896338.523.1419230918.308_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,017/pet0/1.2.840.113619.2.354.3.3523896338.523.1419230918.308_nifti_mask.nii',
 '3',
 '8.19']

In [17]:
for row in csv_data_2 : 
    for ligne in csv_data : 
        if row[1] in ligne : 
            row.append(ligne[4])

In [18]:
csv_data_2[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 'pet0',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_mask.nii',
 '20',
 '9.66',
 'cr_014_jul 25_2014.csv']

In [23]:
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

with open(os.path.join(nifti_directory, 'FLIP_metrics.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_NAME", "PATIENT_ID", "STUDY_UID", "CSV_NAME", "PET", "NIFTI_CT" ,"NIFTI_PET", "NIFTI_MASK", "ROI", "SUV_MAX"])
    for serie in csv_data_2: 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[-1], serie[3], serie[4], serie[5], serie[6], serie[7], serie[8]])